<a href="https://colab.research.google.com/github/piyush-bhandari-commits/self-learn-repo/blob/master/Text_Classification_AutoML_and_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install default-jre
!java -version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
openjdk version "11.0.8" 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)


In [2]:
!pip install h2o

     |████████████████████████████████| 129.3MB 95kB/s 
  Created wheel for h2o: filename=h2o-3.30.1.1-py2.py3-none-any.whl size=129358602 sha256=fb12e4e08da6d730e57d12658285da2a43f49dd7948925f18ed3a532c4cccac9
  Stored in directory: /root/.cache/pip/wheels/33/ac/52/165c35d747abdb629c3c9fb7e087f360c662d8cb58824caed8
Successfully built h2o


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
import h2o
from h2o.automl import H2OAutoML
import seaborn as sns
from pprint import pprint
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.8" 2020-07-14; OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpbwf0h7ai
  JVM stdout: /tmp/tmpbwf0h7ai/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpbwf0h7ai/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,16 days
H2O_cluster_name:,H2O_from_python_unknownUser_325spt
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [5]:
# Fetching the dataset from the data.gov website
url = 'https://files.consumerfinance.gov/ccdb/complaints.csv.zip'
actual_data = pd.read_csv(url, compression='zip')

In [6]:
# Pre-preocessing the actual dataset
print ('Shape of Actual Data: {}'.format(actual_data.shape))

Shape of Actual Data: (1727583, 18)


In [7]:
actual_data.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc. \nis trying to collect...,NaN,TRANSWORLD SYSTEMS INC,FL,335XX,NaN,Consent provided,Web,2019-09-24,Closed with explanation,Yes,NaN,3384392
1,2019-09-19,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,15206,NaN,Consent not provided,Web,2019-09-20,Closed with non-monetary relief,Yes,NaN,3379500
2,2019-10-25,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I would like to request the suppression of the...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,937XX,NaN,Consent provided,Web,2019-10-25,Closed with explanation,Yes,NaN,3417821
3,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,"Over the past 2 weeks, I have been receiving e...",NaN,"Diversified Consultants, Inc.",NC,275XX,NaN,Consent provided,Web,2019-11-08,Closed with explanation,Yes,NaN,3433198
4,2019-02-08,Vehicle loan or lease,Lease,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,NaN,NaN,HYUNDAI CAPITAL AMERICA,FL,33161,NaN,Consent not provided,Web,2019-02-08,Closed with non-monetary relief,Yes,NaN,3146310


In [8]:
raw_dataset = actual_data[[actual_data.columns[1], actual_data.columns[5], actual_data.columns[7]]].rename(columns={'Consumer complaint narrative': 'Complaint'})
clean_raw_dataset = raw_dataset.dropna()
print('The dataset has {} rows and {} columns'.format(clean_raw_dataset.shape[0], clean_raw_dataset.shape[1]))
clean_raw_dataset.head()

The dataset has 585178 rows and 3 columns


,Product,Complaint,Company
0,Debt collection,transworld systems inc. \nis trying to collect...,TRANSWORLD SYSTEMS INC
2,"Credit reporting, credit repair services, or o...",I would like to request the suppression of the...,"TRANSUNION INTERMEDIATE HOLDINGS, INC."
3,Debt collection,"Over the past 2 weeks, I have been receiving e...","Diversified Consultants, Inc."
8,"Money transfer, virtual currency, or money ser...","I was sold access to an event digitally, of wh...","Paypal Holdings, Inc"
9,Mortgage,This complaint dates back to last XX/XX/XXXX d...,NATIONSTAR MORTGAGE


In [9]:
clean_raw_dataset['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    197065
Debt collection                                                                 120326
Mortgage                                                                         68025
Credit card or prepaid card                                                      41344
Credit reporting                                                                 31588
Student loan                                                                     26750
Checking or savings account                                                      23911
Credit card                                                                      18838
Bank account or service                                                          14885
Money transfer, virtual currency, or money service                               10134
Vehicle loan or lease                                                            10047
Consumer Loan                              

In [10]:
dataset = clean_raw_dataset.sample(n=100000)
dataset['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    33816
Debt collection                                                                 20327
Mortgage                                                                        11676
Credit card or prepaid card                                                      7190
Credit reporting                                                                 5382
Student loan                                                                     4550
Checking or savings account                                                      4075
Credit card                                                                      3226
Bank account or service                                                          2511
Money transfer, virtual currency, or money service                               1780
Vehicle loan or lease                                                            1712
Consumer Loan                                         

In [11]:
label_count = dataset['Product'].value_counts().to_dict()
for label, count in label_count.items():
    if count < 3000:
        dataset = dataset.drop(dataset[dataset.Product.values == label].index)

In [12]:
dataset.Product.value_counts()

Credit reporting, credit repair services, or other personal consumer reports    33816
Debt collection                                                                 20327
Mortgage                                                                        11676
Credit card or prepaid card                                                      7190
Credit reporting                                                                 5382
Student loan                                                                     4550
Checking or savings account                                                      4075
Credit card                                                                      3226
Name: Product, dtype: int64

In [13]:
dataset.Company.value_counts()

EQUIFAX, INC.                             11171
Experian Information Solutions Inc.       10403
TRANSUNION INTERMEDIATE HOLDINGS, INC.    10318
CITIBANK, N.A.                             2906
Navient Solutions, LLC.                    2599
                                          ...  
Law Offices of Alan J. Waintraub PLLC         1
Red Cedar Services, Inc                       1
CJA Auto Sales, Inc.                          1
WESBANCO BANK, INC.                           1
Asset Management Holdings, LLC                1
Name: Company, Length: 2592, dtype: int64

In [14]:
mapping = {}
for value in np.arange(len(np.unique(dataset.Product.values))):
    mapping[np.unique(dataset.Product.values)[value]] = value 

In [15]:
mapping

{'Checking or savings account': 0,
 'Credit card': 1,
 'Credit card or prepaid card': 2,
 'Credit reporting': 3,
 'Credit reporting, credit repair services, or other personal consumer reports': 4,
 'Debt collection': 5,
 'Mortgage': 6,
 'Student loan': 7}

In [16]:
dataset['Target'] = dataset.Product.map(mapping)
dataset.head()

,Product,Complaint,Company,Target
1463878,Mortgage,Ocwen is a Servicer on this loan. Loan # XXXX....,Ocwen Financial Corporation,6
947380,Credit reporting,i have many inquiries on my credit file that i...,Experian Information Solutions Inc.,3
1322303,Debt collection,I went to XXXX XXXX ( aka XXXX XXXX ) in XX/XX...,"I.C. System, Inc.",5
675184,Mortgage,To Whom It May Concern : I asked for a mortgag...,WELLS FARGO & COMPANY,6
440075,"Credit reporting, credit repair services, or o...",Ive filed a dispute regarding the results on m...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",4


In [17]:
# Splitting data into train and test set
train, test = train_test_split(dataset, test_size=0.20, random_state=42)

In [18]:
print ('Shape of Train Data : {}'.format(train.shape))
print ('Shape of Test Data : {}'.format(test.shape))

Shape of Train Data : (72193, 4)
Shape of Test Data : (18049, 4)


In [19]:
pprint (train['Complaint'].values[10])

('I had my personal information compromised over a year ago. I have account '
 'and address that are not mine showing on my credit report. Please delete the '
 'following adresses : XXXX XXXX XXXX, XXXX, OH XXXX XXXX XXXX XXXX, XXXX, OH '
 'XXXX My XXXX XXXX XXXX The following phone numbers are not mine ( XXXX ) '
 'XXXX ( XXXX ) XXXX ( XXXX ) XXXX ( XXXX ) XXXX ( XXXX ) XXXX ( XXXX ) XXXX ( '
 'XXXX ) XXXX The following account is not mine as well XXXX XXXX XXXX XXXX # '
 'XXXX I never had a XXXX XXXX XXXX   account. please delete ASAP')


In [20]:
pprint (train['Complaint'].values[28])

('I currently have a mortgage with XX/XX/XXXX. I am a XXXX  in XXXX County. In '
 'XX/XX/XXXX Hurricane Irma struck and we requested a disaster forbearance for '
 'our home. XX/XX/XXXX granted the forbearance to the end of XX/XX/XXXX. At '
 'the time my mortgage payment was {$2600.00} per month. I had never been late '
 'since buying the home in XX/XX/XXXX. They told me at the end of XX/XX/XXXX '
 'to call them to set up new terms for the mortgage. I did so and was put on a '
 '3 month "  trial \'\' period of an increased payment from what I had been '
 'paying prior ( {$2600.00} to {$2700.00} ). At all points in this forbearance '
 'and modification of the loan have I followed all of their requests without '
 'fail. In XX/XX/XXXX they reported to my credit report that I am 60 days '
 'delinquent on my payments thus literally destroying my good credit. I have '
 'requested with them that they fix the credit report with an explanation the '
 'I have been under a forbearance due to disas

In [21]:
pprint (train['Complaint'].values[32])

('My XXXX XXXX and XXXX XXXX is still reporting data that past I past due on '
 'my credit report.The accounts are closed and past the statute of '
 'limitation.Information was submitted to XXXX XXXX to remove the information '
 'off my rercord.')


In [69]:
lemma = WordNetLemmatizer()

porter = nltk.PorterStemmer()

def clean_text(text):
    no_url_text = re.sub(r"http\S+", "", text)
    no_punct_text = re.sub("[^a-zA-Z#]", " ", no_url_text)
    no_num_text = re.sub("\d+", " ", no_punct_text)
    no_encrypt_text = re.sub("^xx$", " ", no_num_text)
    no_small_text = " ".join([word for word in no_encrypt_text.split() if len(word)>2])
    no_stop_text = " ".join([word for word in no_small_text.split() if word not in stopwords])
    # lemma_text = " ".join([lemma.lemmatize(word) for word in no_stop_text.split()])
    stem_text = " ".join([porter.stem(word) for word in no_stop_text.split()])
    tokens = re.split('\W+', stem_text)
    return tokens

train['Complaint Tokens'] = train['Complaint'].apply(lambda x: clean_text(x.lower()))
train['Complaint Sentence'] = train['Complaint Tokens'].apply(lambda x: " ".join(x))
train.head(5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Product,Complaint,Company,Target,Complaint Tokens,Complaint Sentence
1398435,Mortgage,I sold my property and my closing attorney wir...,"CITIZENS FINANCIAL GROUP, INC.",6,"[sold, properti, close, attorney, wire, full, ...",sold properti close attorney wire full heloc p...
506270,Debt collection,Upon careful review of my credit reports I fou...,"Lockhart, Morris & Montgomery Inc.",5,"[upon, care, review, credit, report, found, co...",upon care review credit report found collect a...
1479478,Debt collection,XXXXProvidian National Bank requested a garnis...,JPMORGAN CHASE & CO.,5,"[xxxxprovidian, nation, bank, request, garnish...",xxxxprovidian nation bank request garnish judg...
1652700,"Credit reporting, credit repair services, or o...",On XX/XX/2018 I went to apply for an auto loan...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",4,"[went, appli, auto, loan, dealership, known, x...",went appli auto loan dealership known xxxx xxx...
382456,"Credit reporting, credit repair services, or o...",I HAVE BEEN A VICTIM OF IDENTITY THEFT AND A L...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",4,"[victim, ident, theft, list, fraudul, item, ap...",victim ident theft list fraudul item appear cr...


In [ ]:
# pprint (train['Complaint Sentence'].values[10])

In [ ]:
# pprint (train['Complaint Sentence'].values[28])

In [63]:
x_train = train['Complaint Sentence']
y_train = train['Target']

x_test = test['Complaint']
y_test = test['Target']
x_train.head()

1398435    sold properti close attorney wire full heloc p...
506270     upon care review credit report found collect a...
1479478    xxxxprovidian nation bank request garnish judg...
1652700    went appli auto loan dealership known xxxx xxx...
382456     victim ident theft list fraudul item appear cr...
Name: Complaint Sentence, dtype: object

In [70]:
tfidf_vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1,2))
x_train_vect = tfidf_vectorizer.fit_transform(x_train)
x_test_vect = tfidf_vectorizer.fit_transform(x_test)

In [71]:
x_train_vect.shape, x_test_vect.shape

((72193, 1000), (18049, 1000))

In [76]:
df_train = pd.DataFrame(x_train_vect.toarray(), columns=tfidf_vectorizer.get_feature_names())
df_train = pd.concat([df_train, train['Target'].reset_index(drop=True)], axis=1)

In [78]:
df_test = pd.DataFrame(x_test_vect.toarray(), columns=tfidf_vectorizer.get_feature_names())
df_test = pd.concat([df_test, test['Target'].reset_index(drop=True)], axis=1)

In [79]:
h2o_train_df = h2o.H2OFrame(df_train)
h2o_test_df = h2o.H2OFrame(df_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [80]:
h2o_train_df['Target'] = h2o_train_df['Target'].asfactor()
h2o_test_df['Target'] = h2o_test_df['Target'].asfactor()

In [81]:
auto_ml = H2OAutoML(max_models=5, seed=10, exclude_algos=["StackedEnsemble"], verbosity="info", nfolds=0, balance_classes=True, max_after_balance_size=0.30)

In [82]:
x = tfidf_vectorizer.get_feature_names()
y = 'Target'

In [ ]:
auto_ml.train(x=x, y=y, training_frame=h2o_train_df, validation_frame=h2o_test_df)

AutoML progress: |
06:22:24.900: Project: AutoML_20200827_62224896
06:22:24.901: Cross-validation disabled by user: no fold column nor nfolds > 1.
06:22:24.908: Setting stopping tolerance adaptively based on the training frame: 0.0037217950616253884
06:22:24.908: Build control seed: 10
06:22:24.908: Since cross-validation is disabled, and no leaderboard frame was provided, automatically split the training data into training and leaderboard frames in the ratio 90/10
06:22:31.430: training frame: Frame key: automl_training_py_1_sid_a093    cols: 1001    rows: 64943  chunks: 83    size: 52538593  checksum: -1375517088547202704
06:22:31.434: validation frame: Frame key: py_2_sid_a093    cols: 1001    rows: 18049  chunks: 24    size: 25303125  checksum: 5244679806193662868
06:22:31.734: leaderboard frame: Frame key: automl_leaderboard_py_1_sid_a093    cols: 1001    rows: 7250  chunks: 83    size: 11507692  checksum: -458145588117324140
06:22:31.734: blending frame: NULL
06:22:31.734: respon